**----------------------------------------------------------------------------------------------------**
# **DATA EXPLORATION**
**----------------------------------------------------------------------------------------------------**

The **data_explorer.py** file is a personal python script in which I use to perform initial exploration of datasets. It uses a function to give key summary features of common data file types. More details on this can be found in the Dataset_Explorer Repository on my Github profile.

In [1]:
# IMPORT DATA EXPLORER PYTHON FILE TO ANALYSE EACH DATASET

import data_explorer

In [2]:
data_explorer.analyze_dataset('data_extracts/accounts.csv')

Total Columns,Total Rows,Duplicate Rows,Memory Usage
7,200,0,"11,328"


,Column Name,Data Type,Null Count,NA Count,Unique Count,Mean,Max,Min,Special Characters Count,Whitespace Count,Break Line Count
0,account_id,int64,0,0,200,100.5,200.0,1.0,0,0,0
1,company_name,object,0,0,200,nan,nan,nan,65,0,0
2,company_address,object,0,0,200,nan,nan,nan,0,0,200
3,contact_person,object,0,0,200,nan,nan,nan,0,0,0
4,contact_phone,object,0,0,200,nan,nan,nan,151,0,0
5,gender,object,0,0,2,nan,nan,nan,0,0,0
6,joining_date,object,0,0,179,nan,nan,nan,200,0,0


account_id,company_name,company_address,contact_person,contact_phone,gender,joining_date
168,Brock and Sons,"1531 Martin Center\nVargaston, IN 16753",Denise Moreno,926-621-0713,F,2019-05-02
46,Thompson-Schmidt,"88204 Wolf Lane Suite 478\nLake Jonathan, ND 72312",Valerie Gutierrez,(390)134-6440,F,2017-05-14
84,Grimes LLC,"703 Laura Tunnel Apt. 251\nLake Michael, WI 83007",Terry Cook,(569)616-4419x42573,M,2018-12-02
125,"Carney, Strickland and Ayala","88387 Jeremy Vista Apt. 995\nWest Sabrina, MI 93748",Austin Flowers,001-314-779-2092x2653,M,2017-10-28
93,Frye-Frank,"35783 Stewart Views\nBreannamouth, MN 83679",Veronica Schneider,9958273819,F,2017-09-26
120,Nelson PLC,"30080 Ward Radial Suite 976\nMeyerland, FL 79640",Travis Lewis,+1-458-297-5388,M,2018-05-09
153,Perez-Villegas,"555 Kevin Ford\nKathrynport, NM 75119",Andrew Gonzales,001-670-935-0102x874,M,2017-04-20
75,Bryant-Williams,"23037 Teresa Crossing Suite 222\nNataliestad, NE 87549",Michael Collins,001-313-050-2654,M,2016-06-11
182,Schwartz and Sons,"289 Kimberly Mount\nSouth Jessicaberg, WY 47971",Dr. Carolyn Mendoza,+1-068-759-4256,F,2018-01-02
2,Downs-Simmons,Unit 4570 Box 3823\nDPO AA 82280,Todd Jones,(485)096-9796x7780,M,2017-11-09


In [3]:
data_explorer.analyze_dataset('data_extracts/invoices.csv')

Total Columns,Total Rows,Duplicate Rows,Memory Usage
4,"9,685",0,"310,048"


,Column Name,Data Type,Null Count,NA Count,Unique Count,Mean,Max,Min,Special Characters Count,Whitespace Count,Break Line Count
0,invoice_id,int64,0,0,"9,685","4,843.0","9,685.0",1.0,0,0,0
1,account_id,int64,0,0,200,98.4,200.0,1.0,0,0,0
2,date_issued,object,0,0,366,nan,nan,nan,"9,685",0,0
3,payment_dates,object,0,0,394,nan,nan,nan,"9,685",0,0


invoice_id,account_id,date_issued,payment_dates
6493,129,2020-10-20,2020-12-13
4001,81,2020-08-12,2020-09-10
8687,178,2020-09-16,2020-10-15
6993,143,2020-09-14,2020-10-14
313,8,2020-05-24,2020-06-18
1361,26,2020-08-17,2020-09-16
6116,121,2020-08-31,2020-10-21
4449,89,2021-03-04,2021-04-24
8967,183,2020-06-10,2020-07-09
3624,75,2020-09-15,2020-10-11


In [4]:
data_explorer.analyze_dataset('data_extracts/invoice_line_items.csv')

Total Columns,Total Rows,Duplicate Rows,Memory Usage
3,"53,481",0,"1,283,672"


,Column Name,Data Type,Null Count,NA Count,Unique Count,Mean,Max,Min,Special Characters Count,Whitespace Count,Break Line Count
0,invoice_id,int64,0,0,"9,685","4,847.1","9,685.0",1.0,0,0,0
1,item_id,object,0,0,100,nan,nan,nan,"53,481",0,0
2,quantity,int64,0,0,4,2.5,4.0,1.0,0,0,0


invoice_id,item_id,quantity
8491,1-320-98453-3,1
8222,0-493-95641-7,3
5428,0-289-91901-0,1
1369,1-67777-184-4,2
5031,0-596-89331-0,4
5388,0-596-89331-0,1
3888,0-469-21982-3,1
2691,0-667-81667-4,4
776,1-65593-056-7,4
9137,1-67777-184-4,2


In [5]:
data_explorer.analyze_dataset('data_extracts/skus.csv')

Total Columns,Total Rows,Duplicate Rows,Memory Usage
5,100,0,"4,128"


,Column Name,Data Type,Null Count,NA Count,Unique Count,Mean,Max,Min,Special Characters Count,Whitespace Count,Break Line Count
0,item_id,object,0,0,100,nan,nan,nan,100,0,0
1,item_name,object,0,0,100,nan,nan,nan,59,0,0
2,item_description,object,0,0,100,nan,nan,nan,63,0,0
3,item_cost_price,float64,0,0,100,"4,871.6","9,956.5",115.7,0,0,0
4,item_retail_price,float64,0,0,100,"10,149.0","28,111.4",239.7,0,0,0


item_id,item_name,item_description,item_cost_price,item_retail_price
1-4274-3175-2,Front-line 4thgeneration solution,synergize granular infrastructures,9868.44,16035.35
0-667-81667-4,Customizable holistic installation,deploy cross-media e-markets,6808.85,8448.28
0-01-717479-1,Integrated needs-based complexity,scale turn-key infrastructures,3425.02,5118.67
1-107-39856-8,Secured transitional open architecture,transition killer mindshare,8675.24,21728.93
0-476-73108-9,Implemented incremental knowledge user,monetize clicks-and-mortar markets,8949.56,12654.14
1-60642-190-5,Enhanced uniform methodology,strategize e-business partnerships,4100.31,10595.29
0-383-11312-1,Reactive object-oriented alliance,syndicate proactive action-items,1170.35,2222.55
0-493-95641-7,Up-sized national Graphical User Interface,embrace web-enabled relationships,4176.51,7546.17
0-14-678009-4,Organic well-modulated system engine,e-enable end-to-end networks,2599.90,6851.41
1-222-32214-5,Re-engineered maximized artificial intelligence,unleash synergistic markets,3830.25,4362.38


**----------------------------------------------------------------------------------------------------**
# **PYSPARK ETL Process**
**----------------------------------------------------------------------------------------------------**

In [6]:
# STEP 1: IMPORT REQUIRED LIBRARIES

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
import warnings

warnings.filterwarnings("ignore")

In [7]:
# STEP 2: CREATE SPARK SESSION

spark = SparkSession.builder.appName("InvoiceETL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/25 10:38:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# STEP 3: LOAD CSV EXTRACTS INTO DATAFRAMES

accounts_df = spark.read.csv("data_extracts/accounts.csv", inferSchema=True, header=True, quote='"', escape="\\", multiLine=True) # USE QUOTE AND MULTILINE FOR company_address
skus_df = spark.read.csv("data_extracts/skus.csv", header=True, inferSchema=True)
invoices_df = spark.read.csv("data_extracts/invoices.csv", header=True, inferSchema=True)
invoice_line_items_df = spark.read.csv("data_extracts/invoice_line_items.csv", header=True, inferSchema=True)

In [10]:
# STEP 4: CREATE TEMP TABLES FOR SPARKSQL

accounts_df.createOrReplaceTempView("accounts")
skus_df.createOrReplaceTempView("skus")
invoices_df.createOrReplaceTempView("invoices")
invoice_line_items_df.createOrReplaceTempView("invoice_line_items")

In [11]:
# STEP 5: TRANSFORM DATAFRAMES INTO FEATURES REQUIRED BY DATA SCIENTISTS

# STEP 5A: CREATE INTERMEDIATE MODEL TO AGGREGATE INVOICE LINE ITEMS

try: 
  
  invoices_total = spark.sql("""

----------------INTERMEDIATE MODEL 'INVOICES_TOTAL' CALCULATES AGGREGATED DATA FOR EACH INVOICE-----------------
---------------------JOINS skus (Stock Keeping Units) INFORMATION INTO invoice_line_items-----------------------
----------------------------THE MODEL IS CREATED TO BE USED LATER IN THE MAIN QUERY-----------------------------

  CREATE OR REPLACE TEMPORARY VIEW invoices_total AS (
      SELECT
        l.invoice_id
        ,SUM(l.quantity) AS total_quantity
        ,ROUND(SUM(s.item_cost_price * l.quantity), 2) AS total_cost_price
        ,ROUND(SUM(s.item_retail_price * l.quantity), 2) AS total_retail_price
          
      FROM invoice_line_items l
        LEFT JOIN skus s
        ON l.item_id = s.item_id

      GROUP BY l.invoice_id
      ORDER BY l.invoice_id )

    """)

  spark.sql("SELECT * FROM invoices_total").show()

except Exception as e:
    # RETRIEVE SQL ERROR MESSAGE
    sql_error_message = str(e)
    print("SQL Error Message:", sql_error_message)



+----------+--------------+----------------+------------------+
|invoice_id|total_quantity|total_cost_price|total_retail_price|
+----------+--------------+----------------+------------------+
|         1|            16|        51850.56|         150688.82|
|         2|             6|         33415.9|          69363.78|
|         3|            15|        74707.57|         147434.99|
|         4|            10|        55635.26|         136826.63|
|         5|            21|        132225.4|         308846.37|
|         6|             8|          4180.4|           9787.72|
|         7|            16|       101173.97|         147924.41|
|         8|            15|        62209.29|         137621.43|
|         9|             5|         6072.95|           11299.5|
|        10|            12|        53269.47|         124059.65|
|        11|            17|         56404.9|          111812.0|
|        12|             3|        13317.12|          35051.03|
|        13|            12|        71612

In [12]:
# STEP 5B: FINAL DATAFRAME USING INTERMEDIATE MODEL AND ADDITIONAL CALCULATIONS


try:
  with warnings.catch_warnings():
        warnings.simplefilter("ignore")
  
  invoices_final = spark.sql("""

------------SELECTS ATTRIBUTES RELATED TO INTERMEDIATE MODEL, INVOICES AND ACCOUNTS-------------
-----------------------------USES invoices AS STARTIUNG POINT-----------------------------------
------------------JOINS invoices_total AND accounts INFORMATION INTO invoices-------------------

  SELECT 
    i.invoice_id AS inv_id
    ,i.account_id AS acct_id
    ,t.total_retail_price AS inv_total
    ,t.total_quantity AS inv_items

    ,DATEDIFF(day, a.joining_date, i.date_issued) AS acct_age       --CALCULATE THE ACCOUNT AGE
    
    ,COUNT(i.invoice_id) OVER (PARTITION BY i.account_id 
      ORDER BY i.date_issued 
      RANGE BETWEEN 120 PRECEDING AND 1 PRECEDING) AS num_inv_120d  --COUNT INVOICES IN THE LAST 120 DAYS FROM DATE ISSUED
    
    ,ROW_NUMBER() OVER (PARTITION BY i.account_id 
      ORDER BY i.date_issued, i.invoice_id) AS cum_tot_inv_acct     --CALCULATE CUMULATIVE INVOICES FOR ACCOUNT

    ,CASE 
      WHEN DATEDIFF(i.payment_dates, i.date_issued) > 30 THEN 1
      ELSE 0
     END AS is_late                                                 --BINARY FLAG IF THE INVOICE IS PAID LATE

  FROM invoices i
    LEFT JOIN invoices_total t
    ON i.invoice_id = t.invoice_id

    LEFT JOIN accounts a
    ON a.account_id = i.account_id

  ORDER BY i.invoice_id


  """)

  invoices_final.show()

except Exception as e:
    # RETRIEVE SQL ERROR MESSAGE
    sql_error_message = str(e)
    print("SQL Error Message:", sql_error_message)




+------+-------+---------+---------+--------+------------+----------------+-------+
|inv_id|acct_id|inv_total|inv_items|acct_age|num_inv_120d|cum_tot_inv_acct|is_late|
+------+-------+---------+---------+--------+------------+----------------+-------+
|     1|      1|150688.82|       16|    1626|          23|              56|      1|
|     2|      1| 69363.78|        6|    1433|          17|              22|      0|
|     3|      1|147434.99|       15|    1392|          12|              13|      0|
|     4|      1|136826.63|       10|    1541|          21|              41|      0|
|     5|      1|308846.37|       21|    1600|          20|              50|      0|
|     6|      1|  9787.72|        8|    1424|          17|              20|      0|
|     7|      1|147924.41|       16|    1402|          16|              17|      0|
|     8|      1|137621.43|       15|    1454|          18|              26|      0|
|     9|      1|  11299.5|        5|    1396|          14|              15| 

In [13]:
# STEP 6: SAVE FINAL DATASET TO CSV FILE

invoices_final.write.option("header",True) \
                    .option("delimiter",",") \
                    .csv("data_output", header=True, mode="overwrite")

In [14]:
# STEP 7: STOP SPARK SESSION

spark.stop()

**----------------------------------------------------------------------------------------------------**
# **DATA MODELS & DESIGN CHOICES**
**----------------------------------------------------------------------------------------------------**

**Data Model:**

- ETL pipeline built using Spark SQL, intermediate data models created to store the results of different stages of data transformation.

- **Intermediate Model:**
    Invoices_total - aggregated summary of invoice items, total quantity of items, total cost price and total retail price for each invoice. The model simplifies the ETL process and provides a cleaner workflow.

    Additional intermediate models may be used in more complex ETL pipelines to enhance performance and enable data validation.


**Design Choices and Scaling Considerations:**

- **Spark SQL:** Using Spark for the ETL pipeline allows us to process large volumes of data efficiently, thanks to its distributed processing capabilities.

- **Scalability:** By partitioning and distributing data across a cluster, Spark can process data in parallel, making it scalable to handle big data workloads.

- **Data Source and Data Lakes:** The pipeline reads data from CSV files as the data source and writes the final features dataset to a CSV file as the data lake. Depending on the production environment, we might use other data sources and lakes like databases and cloud storage.

- **Testing the ETL Job:** To test the ETL job, we can use unit testing frameworks like PySpark's pyspark.sql.test.DataFrameTest, which allows us to compare expected results with the ETL pipeline's output.